In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertModel

/home/kihara/nibtehaz/.conda/envs/Almost3dUnet/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [87]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [104]:
model.encoder.layer[6:]

ModuleList(
  (0-5): 6 x BertLayer(
    (attention): BertAttention(
      (self): BertSdpaSelfAttention(
        (query): Linear(in_features=768, out_features=768, bias=True)
        (key): Linear(in_features=768, out_features=768, bias=True)
        (value): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (output): BertSelfOutput(
        (dense): Linear(in_features=768, out_features=768, bias=True)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (intermediate): BertIntermediate(
      (dense): Linear(in_features=768, out_features=3072, bias=True)
      (intermediate_act_fn): GELUActivation()
    )
    (output): BertOutput(
      (dense): Linear(in_features=3072, out_features=768, bias=True)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [95]:
encoded_input['attention_mask'].dtype

torch.int64

In [90]:
tokenizer.pad_token_type_id

0

In [92]:
tokenizer.decode(0)

'[PAD]'

In [89]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_added_tokens_decoder',
 '_added_tokens_encoder',
 '_auto_class',
 '_batch_encode_plus',
 '_batch_prepare_for_model',
 '_call_one',
 '_convert_id_to_token',
 '_convert_token_to_id',
 '_convert_token_to_id_with_added_voc',
 '_create_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eventual_warn_about_too_long_sequence',
 '_eventually_correct_t5_max_length',
 '_from_pretrained',
 '_get_files_timestamps',
 '_get_padding_truncation_strategies',
 '_in_target_context_manager',
 '_pad',
 '_pad_token_

In [ ]:
from datasets import load_dataset

dataset = load_dataset('leondz/wnut_17',trust_remote_code=True)

label_encoder = LabelEncoder()
all_labels = [label for example in dataset['train'] for label in example['ner_tags']]
label_encoder.fit(all_labels)

In [71]:
train_dataset = WNUTDataset(dataset['train'], label_encoder)


In [77]:
max([len(example['tokens']) for example in dataset['test']])

105

In [43]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [44]:
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [85]:
tokenized_sentence, wrd_lbls, tokens, labels = train_dataset.__getitem__(10)
print(tokenized_sentence)
print(labels)
print(len(tokenized_sentence),len(wrd_lbls),len(tokens),len(labels))

['@suzie55', 'whispering', 'cause', 'i', 'may', 'have', 'had', '1', 'too', 'many', 'vodka', "'s", 'last', 'night', 'and', 'am', 'a', 'lil', 'fragile', ',', 'hold', 'me', '?']
[tensor(0), 13, 13, 13, tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(11), tensor(0), 13, tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0), tensor(0)]
23 23 27 27


In [84]:
torch.tensor(13,dtype=torch.long).dtype

torch.int64

In [83]:
labels[0].dtype

torch.int64

In [86]:
for i in range(len(tokens)):
    print(tokens[i],labels[i],tokenizer.decode(tokens[i]))

1030 tensor(0) @
10514 13 su
14272 13 ##zie
24087 13 ##55
13550 tensor(0) whispering
3426 tensor(0) cause
1045 tensor(0) i
2089 tensor(0) may
2031 tensor(0) have
2018 tensor(0) had
1015 tensor(0) 1
2205 tensor(0) too
2116 tensor(0) many
21092 tensor(11) vodka
1005 tensor(0) '
1055 13 s
2197 tensor(0) last
2305 tensor(0) night
1998 tensor(0) and
2572 tensor(0) am
1037 tensor(0) a
13451 tensor(0) lil
13072 tensor(0) fragile
1010 tensor(0) ,
2907 tensor(0) hold
2033 tensor(0) me
1029 tensor(0) ?


In [55]:
print(orig_sentence)

['@paulwalk', 'It', "'s", 'the', 'view', 'from', 'where', 'I', "'m", 'living', 'for', 'two', 'weeks', '.', 'Empire', 'State', 'Building', '=', 'ESB', '.', 'Pretty', 'bad', 'storm', 'here', 'last', 'evening', '.']


In [49]:
' '.join(tokenized_sentence)

"@paulwalk it 's the view from where i 'm living for two weeks . empire state building = esb . pretty bad storm here last evening ."

In [60]:
tokenizer.decode(2009)

'it'

In [66]:
max(label_encoder.classes_)

12

In [58]:
tokenizer(tokenized_sentence)['input_ids']

[[101, 1030, 2703, 17122, 102],
 [101, 2009, 102],
 [101, 1005, 1055, 102],
 [101, 1996, 102],
 [101, 3193, 102],
 [101, 2013, 102],
 [101, 2073, 102],
 [101, 1045, 102],
 [101, 1005, 1049, 102],
 [101, 2542, 102],
 [101, 2005, 102],
 [101, 2048, 102],
 [101, 3134, 102],
 [101, 1012, 102],
 [101, 3400, 102],
 [101, 2110, 102],
 [101, 2311, 102],
 [101, 1027, 102],
 [101, 9686, 2497, 102],
 [101, 1012, 102],
 [101, 3492, 102],
 [101, 2919, 102],
 [101, 4040, 102],
 [101, 2182, 102],
 [101, 2197, 102],
 [101, 3944, 102],
 [101, 1012, 102]]

In [70]:
class WNUTDataset(Dataset):
    def __init__(self, data, label_encoder):
        self.data = data
        self.label_encoder = label_encoder
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        # Preprocess data
        self.sentences = [example['tokens'] for example in self.data]
        self.labels = [example['ner_tags'] for example in self.data]
        self.num_classes = max(label_encoder.classes_)
        
        # Tokenize (basic) and encode labels
        self.tokenized_inputs = [[word.lower() for word in sentence] for sentence in self.sentences]
        # Label encoding tokenizer - assign a number to each word in the dataset vocabulary.
        self.encoded_labels = [self.label_encoder.transform(labels) for labels in self.labels]
        
        # Build vocabulary
        # using bert vocab

    def __len__(self):
        return len(self.sentences)

    def adjust_token_and_label(self,tokens_by_wrd,wrd_lbls):

        tokens = []
        labels = []

        for i in range(len(tokens_by_wrd)):
            for j in range(1,len(tokens_by_wrd[i])-1):
                tokens.append(tokens_by_wrd[i][j])
                labels.append(wrd_lbls[i] if j==1 else torch.tensor(self.num_classes+1,dtype=torch.long))

        return tokens, labels

    def __getitem__(self, idx):
        
        tokenized_sentence = self.tokenized_inputs[idx]
        
        bert_tokens_by_wrd = self.tokenizer(tokenized_sentence)['input_ids']
        wrd_lbls = torch.tensor(self.encoded_labels[idx], dtype=torch.long)

        tokens, labels = self.adjust_token_and_label(bert_tokens_by_wrd,wrd_lbls)

        return tokens, labels


In [2]:
!pip3 install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 4.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.9/780.9 kB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.0/462.0 kB 4.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 4.7 MB/s eta 0:00:00a 0:00:01


In [2]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)


In [22]:
tokenizer.ids_to_tokens[2066]

'like'

In [25]:
output['last_hidden_state'].shape,output['pooler_output'].shape

(torch.Size([1, 12, 768]), torch.Size([1, 768]))

In [4]:
encoded_input['input_ids']

tensor([[ 101, 5672, 2033, 2011, 2151, 3793, 2017, 1005, 1040, 2066, 1012,  102]])

In [9]:
text.split(' ')

['Replace', 'me', 'by', 'any', 'text', "you'd", 'like.']

In [8]:
encoded_input['input_ids'].shape

torch.Size([1, 12])

In [6]:
output['last_hidden_state'].shape,output['pooler_output'].shape

(torch.Size([1, 12, 768]), torch.Size([1, 768]))

In [36]:
max(all_labels)

12

In [33]:
label_encoder.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [30]:
from datasets import load_dataset

dataset = load_dataset('leondz/wnut_17',trust_remote_code=True)

label_encoder = LabelEncoder()
all_labels = [label for example in dataset['train'] for label in example['ner_tags']]
label_encoder.fit(all_labels)

LabelEncoder()

LabelEncoder()

In [28]:
from tqdm import tqdm

class WNUTDataset(Dataset):
    def __init__(self, data, label_encoder):
        self.data = data
        self.label_encoder = label_encoder
        
        # Preprocess data
        self.sentences = [example['tokens'] for example in self.data]
        self.labels = [example['ner_tags'] for example in self.data]
        
        # Tokenize (basic) and encode labels
        self.tokenized_inputs = [[word.lower() for word in sentence] for sentence in self.sentences]
        # Label encoding tokenizer - assign a number to each word in the dataset vocabulary.
        self.encoded_labels = [self.label_encoder.transform(labels) for labels in self.labels]
        
        # Build vocabulary
        self.word_to_idx = {word: idx + 1 for idx, word in enumerate({word for sentence in self.tokenized_inputs for word in sentence})}
        self.word_to_idx['<PAD>'] = 0  # Padding token

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        tokenized_sentence = self.tokenized_inputs[idx]
        input_ids = torch.tensor([self.word_to_idx.get(word, 0) for word in tokenized_sentence], dtype=torch.long)
        labels = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return input_ids, labels


n_epochs = 100

class SimpleLogisticRegressionModel(nn.Module):
    def __init__(self, vocab_size, num_labels):
        super(SimpleLogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(vocab_size, num_labels)

    def forward(self, input_ids):
        # Perform one-hot encoding of input_ids
        one_hot_inputs = F.one_hot(input_ids, num_classes=self.linear.in_features).float()
        logits = self.linear(one_hot_inputs)
        return logits

    def train(self, dataset, batch_size=1):
        # Prepare DataLoader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate_fn)
        
        # Optimizer and Loss function - feel free to change
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        criterion = nn.CrossEntropyLoss(ignore_index=-1)
        
        # Training loop
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        
        
        for epc in range(n_epochs):
            total_loss = 0
            for input_ids, labels in tqdm(dataloader):
                input_ids, labels = input_ids.to(device), labels.to(device)

                optimizer.zero_grad()
                logits = self(input_ids)
                loss = criterion(logits.view(-1, logits.size(-1)), labels.view(-1))
                loss.backward()
                optimizer.step()

                total_loss += loss.item()
            print(total_loss)

        return self

    def predict(self, dataset, batch_size=16) -> list[list[int]]:
        """
        Inference logic for NER task
        """
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)
        
        predictions = []
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        
        with torch.no_grad():
            # Note: do not use gold labels for the final inference, we will remove it while testing
            for input_ids, _ in tqdm(dataloader):
                input_ids = input_ids.to(device)                
                outputs = self(input_ids)
                
                _, preds = torch.max(outputs, dim=-1)
                """
                Extra credit:
                # optionally can implement another inference on top of the model output
                # e.g. CRF, etc.
                """
                predictions.append(preds.tolist())

        return predictions

    def evaluate(self, dataset, batch_size=16) -> list[list[int]]:
        """
        Inference logic for NER task
        """
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate_fn)
        
        ground_truths = []
        predictions = []
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(device)
        
        with torch.no_grad():
            # Note: do not use gold labels for the final inference, we will remove it while testing
            for input_ids, labels in tqdm(dataloader):
                input_ids, labels = input_ids.to(device), labels.to(device)
                outputs = self(input_ids)
                
                _, preds = torch.max(outputs, dim=-1)
                """
                Extra credit:
                # optionally can implement another inference on top of the model output
                # e.g. CRF, etc.
                """
                ground_truths.append(labels.tolist())
                predictions.append(preds.tolist())

        return ground_truths,predictions

# This function is meant for padding the dataset's examples so that every input is the same length
def pad_collate_fn(batch):
    input_ids, labels = zip(*batch)
    max_len = max(len(ids) for ids in input_ids)
    padded_inputs = [F.pad(ids, (0, max_len - len(ids)), value=0) for ids in input_ids]
    padded_labels = [F.pad(label, (0, max_len - len(label)), value=-1) for label in labels]
    return torch.stack(padded_inputs), torch.stack(padded_labels)

In [8]:
torch.optim.Adam(model.parameters())

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [48]:
torch.__version__

'2.2.0+cu121'

In [39]:
torch.cuda.is_available()

True

In [29]:
dataset['train'].data['tokens'][0]

NameError: name 'dataset' is not defined

In [27]:
train_dataset = WNUTDataset(dataset['train'], label_encoder)
val_dataset = WNUTDataset(dataset['validation'], label_encoder)
test_dataset = WNUTDataset(dataset['test'], label_encoder) 

NameError: name 'dataset' is not defined

In [58]:
vocab_size = len(train_dataset.word_to_idx) + 1
num_labels = len(label_encoder.classes_)
model = SimpleLogisticRegressionModel(vocab_size, num_labels)

In [59]:
model.train(train_dataset, batch_size=16)

  0%|          | 0/213 [00:00<?, ?it/s]

100%|██████████| 213/213 [00:01<00:00, 129.82it/s]


488.2224552631378


100%|██████████| 213/213 [00:01<00:00, 131.58it/s]


382.9216922521591


...

100%|██████████| 213/213 [00:01<00:00, 153.26it/s]


19.126047799363732


100%|██████████| 213/213 [00:01<00:00, 131.29it/s]

18.81336254067719


SimpleLogisticRegressionModel(
  (linear): Linear(in_features=12842, out_features=13, bias=True)
)

In [47]:
all_lbls = {}

for i in tqdm(range(len(train_dataset))):

    for lbl in list(train_dataset.__getitem__(i)[1].numpy()):
        if lbl not in all_lbls:
            all_lbls[lbl] = 0
        all_lbls[lbl] += 1

100%|██████████| 3394/3394 [00:00<00:00, 9837.22it/s]


In [48]:
all_lbls

{0: 59570,
 7: 548,
 8: 245,
 5: 264,
 1: 221,
 9: 660,
 3: 140,
 11: 142,
 10: 335,
 4: 206,
 2: 46,
 6: 150,
 12: 203}

In [70]:
yt,yp = model.evaluate(val_dataset, batch_size=16)

100%|██████████| 64/64 [00:00<00:00, 240.75it/s]


In [71]:
print(yt[0][0])
print(yp[0][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
